# We try to replace the (exact-match) cache with a smooth Cassandra

In [1]:
from langchain.llms import OpenAI

In [2]:
import langchain

In [3]:
from CassandraCache import CassandraCache

In [4]:
# This is where the session is created to later pass it to the CassandraCache
import os
from dotenv import load_dotenv
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

load_dotenv()

ASTRA_DB_SECURE_BUNDLE_PATH = os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"]
ASTRA_DB_CLIENT_ID = "token"
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]

cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        ASTRA_DB_CLIENT_ID,
        ASTRA_DB_APPLICATION_TOKEN,
    ),
)
astraSession = cluster.connect()

In [5]:
langchain.llm_cache = CassandraCache(session=astraSession, keyspace='langchain')

In [6]:
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

In [7]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm("How many eyes do spiders have?")

CPU times: user 36 ms, sys: 9.74 ms, total: 45.8 ms
Wall time: 929 ms


'\n\nMost spiders have eight eyes, but some have six or fewer.'

In [8]:
%%time
# This time we expect a much shorter answer time
llm("How many eyes do spiders have?")

CPU times: user 1.69 ms, sys: 177 µs, total: 1.87 ms
Wall time: 122 ms


'\n\nMost spiders have eight eyes, but some have six or fewer.'

## You may want to check the Astra table now.